In [31]:
import numpy as np
import matplotlib.pyplot as plt

seed = 42
np.random.seed(seed)

N = 5
time_series = [np.random.randint(N)]
print(time_series)

transition_matrix = np.random.rand(N, N)
print(transition_matrix)

# axis 表示“被合并的维度”, axis=1表示沿行方向计算，列被合并
transition_matrix /= transition_matrix.sum(axis=1)[:, np.newaxis]
print(transition_matrix)

current_state = time_series[-1]
print(current_state)

p=transition_matrix[current_state]
print(p)

# np.random.choice 用于从一个 1D 数组中随机选择元素（可以重复或不重复），生成一个新的随机样本
# 整数N → 表示从 [0, 1, ..., N-1] 中选
next_state = np.random.choice(N, p=p)
print(next_state)

[3]
[[0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501 0.70807258 0.02058449]
 [0.96990985 0.83244264 0.21233911 0.18182497 0.18340451]
 [0.30424224 0.52475643 0.43194502 0.29122914 0.61185289]
 [0.13949386 0.29214465 0.36636184 0.45606998 0.78517596]]
[[0.36659277 0.28225481 0.23084103 0.06016035 0.06015105]
 [0.02576876 0.38427858 0.26668435 0.31413601 0.0091323 ]
 [0.40753866 0.34977741 0.08922107 0.07639958 0.07706327]
 [0.14059086 0.24249085 0.19960253 0.13457749 0.28273827]
 [0.06840462 0.14326109 0.17965551 0.22364635 0.38503243]]
3
[0.14059086 0.24249085 0.19960253 0.13457749 0.28273827]
1


In [32]:
for _ in range(N - 1):
    current_state = time_series[-1]
    next_state = np.random.choice(N, p=transition_matrix[current_state])
    time_series.append(next_state)
time_series

[3, 2, 1, 1, 2]

In [ ]:
a = np.array([[1,2,3],
              [4,5,6]], dtype=float)
b = a.sum(axis=1)[:, np.newaxis]
a /= b
a

array([[0.16666667, 0.33333333, 0.5       ],
       [0.26666667, 0.33333333, 0.4       ]])

- Zipf定律是指取值的排名和频率成反比，排名是根据取值频率得到的
- Zipf分布生成的转移矩阵，向其中一个状态转移的概率远大于向剩余状态转移

In [33]:
from scipy.stats import zipf

In [38]:
zipf_dist = zipf(2)
zipf_dist


In [47]:
np.random.seed(seed)
values = zipf_dist.rvs(size=(N, N))
values

array([[1, 3, 1, 1, 2],
       [1, 5, 1, 1, 1],
       [2, 1, 1, 1, 2],
       [2, 1, 1, 3, 1],
       [1, 1, 1, 2, 1]], dtype=int64)

In [ ]:
normalized_values = values / np.max(values) 
normalized_values # normalized_values: [0,1]

array([[0.2, 0.6, 0.2, 0.2, 0.4],
       [0.2, 1. , 0.2, 0.2, 0.2],
       [0.4, 0.2, 0.2, 0.2, 0.4],
       [0.4, 0.2, 0.2, 0.6, 0.2],
       [0.2, 0.2, 0.2, 0.4, 0.2]])

In [ ]:
normalized_matrix = normalized_values / normalized_values.sum(axis=1, keepdims=True)
normalized_matrix # 每一行加起来为1

array([[0.125     , 0.375     , 0.125     , 0.125     , 0.25      ],
       [0.11111111, 0.55555556, 0.11111111, 0.11111111, 0.11111111],
       [0.28571429, 0.14285714, 0.14285714, 0.14285714, 0.28571429],
       [0.25      , 0.125     , 0.125     , 0.375     , 0.125     ],
       [0.16666667, 0.16666667, 0.16666667, 0.33333333, 0.16666667]])

In [53]:
import pandas as pd

In [101]:
N = 5
n = 50000
a = 15
length = 20000

In [102]:
input_path = "../Datasets/Markov/Exp1/markov_ts_"+str(N)+"_"+str(n)+"_"+str(a)+".csv"

In [103]:
def get_data(path, length):
    df = pd.read_csv(path)
    df = df.iloc[:length, :]
    return df.iloc[:, 0].values 

In [104]:
df = pd.read_csv(input_path, header=None)
df.head()

,0
0,1
1,0
2,3
3,1
4,4


- length的取值影响H

In [105]:
df = df.iloc[:length,:]
df

,0
0,1
1,0
2,3
3,1
4,4
...,...
19995,0
19996,0
19997,0
19998,1


In [106]:
series = df.iloc[:, 0].values 

In [107]:
from LZ2 import * 
from compute_PIMax import * 

In [108]:
def maximum_predictability(N, S):
    """ The Maximum Predictability function
    Input:
        N: number of unique values in time series, integer
        S: value of entropy, float
    Output:
        value of the Maximum Predictability 
        or 'No solutions' if it cannot be determined
    """
    
    def Function(x, N, S):
        """ The Maximum Predictability function
        """
        return 1.0*(-x*np.log(x)-(1-x)*np.log(1-x)+(1-x)*np.log(N-1)-S*np.log(2))

    def FirstDerivative(x, N):
        """ First derivative of the function
        """
        return 1.0*(np.log(1-x)-np.log(x)-np.log(N-1))

    def SecondDerivative(x):
        """ Second derivative of the function
        """
        return 1.0/((x-1)*x)
    
    def CalculateNewApproximation(x, N, S):
        """ One iteration of Householder's method
            of second order
        """
        function = Function(x, N, S)
        first_derivative = FirstDerivative(x, N)
        second_derivative = SecondDerivative(x)
        return 1.0*function/(first_derivative-function*second_derivative/(2*first_derivative))
 
    S = round(S, 9)
    if S > round(np.log2(N), 9):
        return "No solutions"
    else:
        if S <= 0.01:
            return 0.999
        else:
            x = (N + 1) / (2 * N)
            while abs(Function(x, N, S)) > 1e-8:
                x = x - CalculateNewApproximation(x, N, S)
    return round(x, 10)

In [109]:
H = Compute_LZ2(series[:int(0.8*len(series))],0.0)
print(maximum_predictability(N, H))

KeyboardInterrupt: 

In [110]:
# series[:int(0.8*len(series))] 前 80% 数据集
H = Compute_LZ2(series[:int(0.8*len(series))],0.0)
pimax = get_pimax(H,N)
print(pimax)

0.3489982849787224
